# Data Exploration
- Counting null values in each column and dropping the six columns containing only null values decreased the proportion of nulls in the dataset from 21.1% to 13.6%.

- Explanations for null values may lie in exploring differences between cohorts, sex, or behavioral environments.

- Males and females contributed similarly to total null values; sex does not seem to be the best avenue to pursue at first glance.

- Cohorts 16 and 17 differ from other cohorts in that they each contribute to more than 10% of total null values in the dataset. However, further comparing against the proportion of total animals reveals that cohorts 4 and 16 contribute 1.6 null values per capita, and cohorts 2 and 17 contribute 1.5 null values per capita.

- Similarly, room BSB273B contributes to 23.6% of total null values in the dataset, but it also was used to run 23.5% of all animals in the dataset, so it only contributes to 1.0 null values per capita.

- The top three rooms by null values per capita ran too few animals to draw meaningful conclusions; excluding these, only room MTF134B had a higher null count per capita than BSB273B. Comparing room number against cohort number reveals that 75% of cohort 16 was run in BSB273B, while 100% of cohort 17 was run in MTF134B. The computers in these rooms may have had issues exporting data from Med Associates software because of how the MPC2XL export template was written, or they may have been using MedState Notation programs that were not written to record all the same variables recorded on other computers.

- Condensing the dataset to the most utile columns and re-inspecting null values added some support to exploring cohort and room differences. For the remaining dataset, columns relating to aggressive/defensive behavior counts contributes to approximately 33% of all null values.

- An easy first step to exploring this further would be confirming which animals were excluded from the aggressive/defensive behavior count aspect of the project, and why. If the number of animals excluded does not correlate with the spread of null values, there may be another issue at play for the missing data.

In [ ]:
# @title # Load CSV into Pandas DataFrame
import pandas as pd
import numpy as np
df_phenotypes = pd.read_csv('https://raw.githubusercontent.com/eliwagnercode/PalmerLab/main/phenotype_data_edited.csv')
df_phenotypes

In [ ]:
# @title # DRY functions to count nulls

def get_nullsByCol(df):
  colNulls = df.isna().sum()
  df_colNulls = colNulls.to_frame(name='n_nulls')\
    .rename_axis('column_name')\
    .sort_values('n_nulls',ascending=False)
  print('Total null count in DataFrame: ',df_phenotypes.isna().sum().sum())
  print('Percent null in DataFrame: ',
      round(100*df.isna().sum().sum()\
      /(df.shape[0]*df.shape[1]),1))
  return df_colNulls

# This function would be more efficient using only numpy arrays, but 
# it works well for a dataset with fewer than 1000 rows.
def get_nullsByGroup(df,column_name_str):
  df['n_nulls'] = df.isna().sum(axis=1)
  df_valueCounts = df.value_counts(column_name_str)\
    .to_frame(name='n_animals')\
    .rename_axis(column_name_str)\
    .reset_index()

  df_valueCounts['total_values'] = df_valueCounts['n_animals']\
    * df.shape[1]

  df_nulls = df.groupby(column_name_str)['n_nulls'].sum()\
    .to_frame(name='n_nulls')\
    .rename_axis(column_name_str)\
    .reset_index()
    
  df_nulls = df_nulls.merge(
      df_valueCounts,on=column_name_str)

  percent_group_values = []
  percent_all_animals = []
  percent_all_nulls = []
  nulls_per_capita = []
  n_animals = df_nulls['n_animals'].to_list()
  nulls = df_nulls['n_nulls'].to_list()
  totals = df_nulls['total_values'].to_list()
  for null_count, animal_count, total_count in zip(nulls,n_animals,totals):
    pct_grp_val = round(100*null_count/total_count,1)
    pct_animals = round(100*animal_count/len(df),1)
    pct_nulls = round(100*null_count/df.copy().isna().sum().sum(),1)
    nulls_per_cap = round(pct_nulls/pct_animals,1)
    percent_group_values.append(pct_grp_val)
    percent_all_animals.append(pct_animals)
    percent_all_nulls.append(pct_nulls)
    nulls_per_capita.append(nulls_per_cap)

  df_nulls['percent_group_values'] = percent_group_values
  df_nulls['percent_all_animals'] = percent_all_animals
  df_nulls['percent_all_nulls'] = percent_all_nulls
  df_nulls['nulls_per_capita'] = nulls_per_capita

  df_nulls = df_nulls[[column_name_str,
                       'n_animals',
                       'percent_all_animals',
                       'n_nulls',
                       'percent_group_values',
                       'percent_all_nulls',
                       'nulls_per_capita'
                       ]]

  df_nulls = df_nulls.sort_values(by='percent_group_values',ascending=False)
  return df_nulls

In [ ]:
# @title Count nulls by column
df = get_nullsByCol(df_phenotypes)
df[:10]

In [ ]:
# @title Drop null-only columns from DataFrame and count nulls by column
colNullsAll = df.loc[
    df['n_nulls'] == len(df_phenotypes)
    ].index.to_list()
df_phenotypes = df_phenotypes.copy().drop(
    columns=colNullsAll)
df = get_nullsByCol(df_phenotypes)
df[:10]

In [ ]:
# @title Count nulls by cohort
df_cohort = get_nullsByGroup(df_phenotypes,'cohort')
df_cohort

In [ ]:
# @title Count nulls by sex
df_sex = get_nullsByGroup(df_phenotypes,'sex')
df_sex

In [ ]:
# @title Count nulls by behavior room
df_room = get_nullsByGroup(df_phenotypes,'room')
df_room

In [ ]:
# @title Count animals by cohort run in room BSB273B
df_phenotypes[['rfid','cohort','room','box']].loc[(df_phenotypes['room']=='BSB273B')].value_counts('cohort')

In [ ]:
# @title Count animals by cohort run in room MTF134B
df_phenotypes[['rfid','cohort','room','box']].loc[(df_phenotypes['room']=='MTF134B')].value_counts('cohort')

In [ ]:
# @title Count nulls by behavior room and operant chamber
df_phenotypes['room_box'] = df_phenotypes['room'].astype(str) + '_' + df_phenotypes['box'].astype(str)
df_room_box = get_nullsByGroup(df_phenotypes,'room_box')
df_phenotypes = df_phenotypes.drop(columns='room_box')
df_room_box

In [ ]:
# @title Combine and drop columns
df_phenotypes['sha_mean_total_presses_08_10'] = df_phenotypes[['sha_mean_08_10', 'sha_mean_inactive_08_10',
       'sha_mean_to_08_10']].sum(axis=1)
df_phenotypes['sha_mean_total_presses_01_03'] = df_phenotypes[['sha_mean_01_03', 'sha_mean_inactive_01_03',
       'sha_mean_to_01_03']].sum(axis=1)
df_phenotypes = df_phenotypes[[
    'cohort', 'rfid', 'sex', 'room','box',
    'irr_age','lga_12_age','pr_01_age','pr_02_age','pr_03_age','sha_01_age','sha_08_age','shock_03_age',
    'sha_mean_01_03','sha_mean_total_presses_01_03','sha_mean_08_10','sha_mean_total_presses_08_10','sha_behavior_stability_08_10', 'sha_behavior_stability_08_10_na','sha_coefficient_of_variation', 'sha_coefficient_of_variation_2',
    'lga_mean_12_14','lga_behavior_stability_12_14', 'lga_behavior_stability_12_14_na','lga_coefficient_of_variation', 
    'shock_03', 'shock_03_pre', 'shock_03_avg1h', 
    'pr_01_sha_breakpoint', 'pr_02_lga_breakpoint','pr_03_postshock_breakpoint', 'pr_max_02_03_breakpoint',
    'irr_agg_change', 'irr_def_change', 'irr_total_change',
    'addiction_index_no_sex_z','add_index_sexcohort_z', 'add_ind_calc_no_z_shock_03','addiction_index_no_sex_z_shock_03_avg1h','add_index_palmer_shock_03_avg1h', 'add_ind_calc_no_z_shock_03_avg1h','add_ind_olivier'
]]

In [ ]:
# @title Count nulls by cohort
df_cohort = get_nullsByGroup(df_phenotypes,'cohort')
df_cohort

In [ ]:
# @title Count null by sex
df_sex = get_nullsByGroup(df_phenotypes,'sex')
df_sex

In [ ]:
# @title Count null by behavior room
df_room = get_nullsByGroup(df_phenotypes,'room')
df_room

In [ ]:
# @title Count null by behavior room and operant chamber
df_phenotypes['room_box'] = df_phenotypes['room'].astype(str) + '-' + df_phenotypes['box'].astype(str)
df_room_box = get_nullsByGroup(df_phenotypes,'room_box')
df_phenotypes = df_phenotypes.drop(columns='room_box')
df_room_box

In [ ]:
df = df_phenotypes.loc[df_phenotypes['room']=='BSB273B']
get_nullsByCol(df)

# Stats

- Because there are so many missing values for traits relating to aggressive/defensive behavior, I decided to only explore self-administration for this assignment.

- Because there are no clearly indicated group differences with respect to genotype, treatment, or training context, the only obvious parameters to run group comparisons for behavioral data are sex and age.

- Given more time for the assignment I might classify animals into age brackets and run two-way ANOVA for different traits against age and sex. Because the maximum age for what is seemingly the last experiment in the project, pr_03_age, is only 143 days, and because there is no indication from the dataset that this strain of animal is a model of age-related disorders, I decided to focus only on exploring sex differences for ShA.

- Outliers were defined as those with values further than 1.5x the IQR away from the first or third quartile. Outliers were only removed before grouping by sex. Upon inspecting within groups, it was revealed that the male group appeared to have a much more condensed spread with more outliers compared to the female group.

- Although the two groups had a different spread, a t-test was not strong enough to reject the null hypothesis.

In [ ]:
print(df_phenotypes.loc[df_phenotypes['isMale']==0]['sha_mean_08_10'].describe())

In [ ]:
df_phenotypes.loc[df_phenotypes['isMale']==1]['sha_mean_08_10'].describe()

In [ ]:
df_phenotypes = df_phenotypes.drop(columns=['irr_age', 'lga_12_age',
       'pr_01_age', 'pr_02_age', 'pr_03_age', 'sha_01_age', 'sha_08_age',
       'shock_03_age','irr_agg_change', 'irr_def_change', 'irr_total_change',])

In [ ]:
# @title Encode sex column with numeric value
encode_sex = []
for sex in df_phenotypes['sex'].to_list():
  if sex == 'F':
    encode_sex.append(0)
  else:
    encode_sex.append(1)
df_phenotypes['isMale'] = encode_sex
df_phenotypes = df_phenotypes.drop(columns='sex')
df_phenotypes

In [ ]:
# @title Inspect ShA for outliers
import seaborn as sns
print(df_phenotypes.loc[df_phenotypes['isMale']==0]['sha_mean_08_10'].describe())
sns.boxplot(df_phenotypes['sha_mean_08_10'])

In [ ]:
# @title Remove outliers by interquartile range approach
Q1 = df_phenotypes['sha_mean_08_10'].quantile(0.25)
Q3 = df_phenotypes['sha_mean_08_10'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR
df_phenotypes = df_phenotypes.loc[
    (df_phenotypes['sha_mean_08_10']<=upper)
    & (df_phenotypes['sha_mean_08_10']>=lower)]

In [ ]:
# @title Divide DataFrame by sex
males = df_phenotypes.loc[df_phenotypes['isMale']==1].reset_index(drop=True)
females = df_phenotypes.loc[df_phenotypes['isMale']==0].reset_index(drop=True)

In [ ]:
# @title Inspect male ShA responding
print(males['sha_mean_08_10'].describe())
sns.boxplot(males['sha_mean_08_10'])

In [ ]:
# @title Inspect female ShA responding
print(females['sha_mean_08_10'].describe())
sns.boxplot(females['sha_mean_08_10'])

In [ ]:
# @title t-test
from scipy.stats import ttest_ind
ttest_ind(males['sha_mean_08_10'],females['sha_mean_08_10'])